yolo 

dark-net --> linux환경에서 사용가능 
darkflow란 것도 있음

In [3]:
import cv2
import numpy as np

min_confidence = 0.5
net = cv2.dnn.readNet("yolov3.weights", "./opencv_dnn_202005/opencv_dnn_202005/yolo/yolov3.cfg")
classes = []
with open("./opencv_dnn_202005/opencv_dnn_202005/yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0,255, size=(len(classes),3))

#loading image
img = cv2.imread("./opencv_dnn_202005/opencv_dnn_202005/image/yolo_01.jpg")
img = cv2.resize (img,None , fx=0.4, fy = 0.4)
height,width, channels = img.shape
cv2.imshow("original Image", img)

#크기가 정해진게 있음 사진 크기에 따라 장단점이 있음 
blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop = False )

net.setInput(blob)
outs = net.forward(output_layers)

#showing informations on the screen
class_ids = []
confidences = []
boxes = [] 

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > min_confidence:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

#NMSBoxes 쓰면 박스모양이 여러개 겹쳐서 나오지 않고 하나만 나옴
indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        print(i, label)
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 2, (0, 255, 0), 1)

cv2.imshow("YOLO Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()




[[11]
 [16]
 [ 0]
 [22]
 [23]
 [ 6]
 [18]
 [13]]
0 car
6 car
11 car
13 person
16 motorbike
18 person
22 person
23 person


In [6]:
#video
import cv2
import numpy as np
import time

min_confidence = 0.5
file_name = './opencv_dnn_202005/opencv_dnn_202005/video/yolo_01.mp4'

def detectAndDisplay(frame):
    #loading image
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    cv2.imshow("Original Image", img)

    #크기가 정해진게 있음 사진 크기에 따라 장단점이 있음 
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop = False )

    net.setInput(blob)
    outs = net.forward(output_layers)

    #showing informations on the screen
    class_ids = []
    confidences = []
    boxes = [] 

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    #NMSBoxes 쓰면 박스모양이 여러개 겹쳐서 나오지 않고 하나만 나옴
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            print(i, label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2.imshow("YOLO Video", img)
    


net = cv2.dnn.readNet("yolov3.weights", "./opencv_dnn_202005/opencv_dnn_202005/yolo/yolov3.cfg")
classes = []
with open("./opencv_dnn_202005/opencv_dnn_202005/yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0,255, size=(len(classes),3))

#read the video stream
cap = cv2.VideoCapture(file_name)
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


0 car: 99.87
3 car: 99.86
4 car: 99.52
5 car: 99.53
9 car: 99.97
11 car: 98.29
16 car: 93.66
=== A frame took 0.911 seconds
0 car: 99.87
3 car: 99.86
4 car: 99.51
5 car: 99.50
9 car: 99.97
11 car: 98.35
15 car: 93.50
=== A frame took 0.571 seconds
0 car: 99.87
3 car: 99.86
4 car: 99.51
5 car: 99.50
9 car: 99.97
11 car: 98.35
15 car: 93.58
=== A frame took 0.584 seconds
0 car: 99.84
1 car: 99.85
2 car: 99.37
4 car: 99.05
7 car: 99.96
9 car: 99.09
11 car: 78.32
=== A frame took 0.531 seconds
0 car: 99.84
1 car: 99.86
2 car: 99.36
3 car: 99.05
6 car: 99.96
8 car: 99.05
10 car: 78.47
=== A frame took 0.528 seconds
0 car: 99.84
1 car: 99.86
2 car: 99.36
4 car: 99.05
7 car: 99.96
9 car: 99.12
11 car: 78.47
=== A frame took 0.522 seconds
0 car: 99.90
1 car: 99.84
3 car: 99.16
5 car: 99.09
9 car: 99.98
11 car: 99.39
22 car: 56.40
=== A frame took 0.532 seconds
0 car: 99.90
1 car: 99.84
3 car: 99.19
5 car: 99.09
9 car: 99.98
11 car: 99.38
22 car: 56.63
=== A frame took 0.522 seconds
0 car: 99.9